# Library Import

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset

In [2]:
from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

2021-09-03 01:30:40.955058: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [4]:
import gluonnlp as nlp

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from tqdm import tqdm, tqdm_notebook
import sys

# Main

In [7]:
%matplotlib inline

In [8]:
plt.style.use('seaborn')
fm.get_fontconfig_fonts()
font_location = '/content/drive/MyDrive/Colab Notebooks/BERT_BMC/font/NanumGothic.ttf'
fprop = fm.FontProperties(fname=font_location)

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import random
random.seed(1004)

In [10]:
num_class = 5

In [11]:
device = torch.device("cuda:0")

# import torch_xla
# import torch_xla.core.xla_model as xm

# net = torchvision.models.alexnet(num_classes=num_class)

# # Acquires the default Cloud TPU core and moves the model to it
# device = xm.xla_device()
# net = net.to(device)

In [12]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model
using cached model
using cached model


In [173]:
# 데이터 로딩
df = pd.read_csv("data/BMC_data.csv")
df

,번호,기업명,고객 세그먼트,점수,가치제안,점수.1,마케팅 채널,점수.2,고객관계,점수.3,수익원,점수.4,핵심활동,점수.5,핵심자원,점수.6,핵심파트너,점수.7,비용구조,점수.8
0,1,에이앤유디자인그룹건축사사무소,"현대건설, 지에스건설 등 건설사\nLH공사, SH공사, 지방 도시공사와 같은 시행사...",A,창의적인 디자인을 통한 풍요로운 도시공간의 가치창출\n건축문화를 선도하는 리더그룹으...,B,"공공기관에서 발주하는 현상설계공모 참여\n건설사, 시행사에서 발주하는 지명경쟁 설계...",D,공공에서 발주하는 경쟁 프로젝트 당선\n디자인을 통한 문제해결의 솔루션 제공,C,설계결과물의 제공을 통한 용역비,D,좋은 디자인\n경쟁공모 프로젝트 당선\n인허가 기간 준수 및 단축,C,"디자인능력과 전문지식을 갖고 있는 설계 인력과 정보, 협상력을 갖춘 영업인력",E,"건설사, 시행사",C,인건비\n외주비\n관리비,E
1,2,멜텍스코리아 주식회사,전자부품MLCC(Multi Layer Ceramic Condenser)제조사\n반도...,A,기술혁신과 환경 부담 개선을 통하여 전자산업 발전에 공헌하고 환경 보전에 힘쓴다. ...,A,국내외 최종 사용자의 승인을 통한 중간 업체 판매 확대\n자체 영업력을 통한 신규 ...,B,지속적인 신기술 소개 및 신제품 제안\n기존 판매 약품에 대한 기술 지원,C,제품 판매 대금 수령\n인쇄회로기판 제조용 표면처리약품 판매 수익\n반도체 제조용 ...,A,국내외 관련 전시회 출품을 통한 제품 소개 및 회사 인지도 확대\n영업인력을 활용한...,B,판매 제품의 높은 품질 신뢰도 및 선진 기술 제공\n전문 영업인력의 기술 서비스에 ...,C,판매 대리점\n창고 및 물류회사\n컨설팅 회사,B,재료비\n인건비\n물류비\n사무실임대료\n기계감가상각비\n영업비,E
2,3,에스티엑스건설,주거용 건물 건축주(입주자아파트 등)\n건설부문 외주 및 자재 생산업체(단종업체) ...,B,"품질, 기술, 완벽한 시공으로 고객만족을 실현\n하자율 최소화(제로)로 입주자 고객...",E,한국토지공사/해양수산부/지자체 등\n신탁회사(개발신탁)/개인 건축주,B,실적을 바탕으로 공공입찰\n신탁회사와 지속적인 정보 공유로 공동주택(재개발/재건축)...,C,경쟁입찰을 통한 계약으로 공사대금 수령\n하자율 최소화로 사후 관리비 적정 투입,C,"관공사, 시행사, 건축주 등 목표고객과 공정 협의를 통해 현장 시공",C,"시공에 필요한 기술자 보유(시공, 공무 등)\n고객만족을 위한 사후관리 시스템",D,"공공 발주처(한국토지공사, 해량수산부, 지자체 등)",A,"인건비, 시공비, 재료비, 하자처리비\n사무실 임대료",E
3,4,충청산업보건연구원(주),"관공서(행정기관)석면사업부(조사, 감리, 측정)\n제조업(대기업)작업환경측정사업부",B,"전문성을 바탕으로 한 신뢰를 구축을 통한 용역서비스 ( 석면조사, 작업환경측정업,...",A,기존거래선의 지속적인 유지 (법령에 의한 의무 용역사업)\n 공공조달(나라장터)의...,C,기업의 전문성과 우수성을 객관적으로 평가를 하는 기관평가(한국산업안전보건공단평가)...,A,기존고객의 지속적인 관계를 통한 용역수행에 대한 용역비용 수령\n 경쟁입찰을 통한...,C,산업안전보건법에 명시된 기업의 근로자를 위한 작업환경측정용역 대행업\n 산업안전보...,A,전문지식을 갖춘 부서별 인적자원\n 법령에 근거한 전문과학기술서비스업\n 오랜 근...,D,"한국산업안전보건공단\n 고용노동부\n 지역내에 존재하는 경쟁회사( 대한산업보건협회,...",A,"인건비\n 설비투자비(분석장비, 측정장비)\n 재료소모품비(시약구입비 등)\n 건...",E
4,5,대동산업(주),"건설사, 시행사, 설계사무소\n도소매 유통업체\n건축주\nLH공사, SH공사, 지방...",C,"고객이 만족하는 제품을 생산하여 공급\n신뢰를 바탕으로 고객, 협력사, 제휴기업과의...",D,"LH공사, SH공사, 지방 관공서 등 직접 영업을 통한 입찰 참가\n공공조달 입찰을...",B,당사 영업부와 디자인연구소가 제품관련 품질 및 디자인 등 니즈를 파악\n목표 고객에...,B,제품 판매로 대금 수령\n공공입찰로 제품 대금 수령\n수의계약을 통한 제품 대금 수령,C,"건설사, 시행사, 등 목표고객에게 신제품 제안\n생산 협의를 통해 적시에 생산하여 ...",C,전문 기술인력과 풍부한 경험을 갖춘 각 부서의 인적자원\n시장 흐름을 파악하여 변화...,B,"건설사 상품개발부서, 설계팀\n설계사무소, 디자인사무소\n공공발주처(LH공사, SH...",C,"인건비, 원재료비, 물류비용\n설비투자비, 기술개발비\n건물 및 기계 감가상각비",C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,500,㈜케이바스,"욕실 용품 유통업체\n시행사, 시공사, 중소 건설사",C,소규모 현장의 고급 월풀욕조의 생산과 시공.\n고급 욕조를 다양하게 취급함으로 고객...,E,거래처의 소개 : 주로 문제 현장에 대한 해결책으로 소개 받음.\n납품했던 제품을 ...,D,제품 특성상 최종 사용자의 다양하고 독특한 요구를 맞출 수 있도록 다양한 방법을 통...,D,욕조 판매 및 시공을 통한 수익,D,"호텔, 펜션, 고급주택에 월풀욕조 및 고급욕조 납품 및 시공",E,월풀 욕조에 특화된 시공 노하우 및 생산기술\n안정적인 해외 수입처,D,대형 유통업체,D,"인건비, 시공비, 재료비, 운영비, 임대료",E
500,501,진우엔티,"노즐 및 밸브를 사용하는 기업\n분수, 아파트 정원 공사업체\n환경설비업을 하는 회...",B,고장 없는 노즐을 공급하여 고객만족을 극대화한다,C,기존 구매했던 고객의 재구매\n인맥을 통한 영업\n당사 영업팀,D,신제품 출시에 맞춰 고객에게 제품 사양을 송부하여 제품 구매 유도\n고객이 기존 기...,D,밸브 판매 수익,D,"설비팀, 공장 기술팀에 당사 제품을 제안하여 기술에 맞는 제품을 제안\n신기술을 적...",D,개발하는 연구인력\n판매 영업 인력,D,"설비회사, 구매부, 발전소 등 고객\nOEM제조회사\n회사별 설비운영팀",D,"인건비, 재료비, 접대비, 홍보비, 임대료, 일반관리비",D
501,502,㈜에어붐,당사 1단계 목표 : 초중고등학교 \n2단계 목표 : B to C 이며 \n3단계...,C,당사의 추구하는 가치 중심은 기술-품질-서비스 순으로 보고 있으며 1차 가치는 혁신...,D,"당사 제품의 특성상 기축시장(기사용중인 양변기 대상)과 신축시장(새로 건축, 리모델...",D,당사는 각 사이트 별로 샘플설치 영업을 우선하고 그 샘플을 고객이 사용해 보게끔 해...,B,당사는 차별화된 기술과 양변기 자동물내림 제품 판매를 통해서 그 수익을 창출하고 있...,B,당사는 R&D 전문 회사로 발전해 가고자 더 다양한 신기술개발에 도전하고 있음.\n...,B,당사는 특화된 기술 자산을 바탕으로 하고 있음. 변기 자동물내림 한 분야에 23년 ...,C,"개발 샘플 제작 업체\n금형 제작 업체\n부품 공급 업체\n위생도기, 비데, 물류회...",D,"개발비, 금형투자비, 생산설비\n인건비, 재료비, \n사무실 임대료, 홍보비 등",B
502,503,㈜킴스코파쎄,"건설사(민영, 공영), 설계사무소, 시행사\n건축자재 도소매 업체, 인테리어 업체\...",C,"품질, 기술, 고객만족을 바탕으로 상호신뢰를 구축하여 지속 가능한 제품 공급\n고객...",C,"민영, 공영 등 시행사 및 시공사 목표 고객에게 직접 영업을 통한 입찰 참가\n홈페...",C,고객의 요청에 따라 목표고객과의 신제품 개발관련 기술협의를 통해 고객의 니즈 파악\...,C,신상품개발을 통해서 이익확대\n자체 원가절감을 통한 판매수익\nB2B에서 B2C를 ...,B,사전 영업을 통한 고객과의 공동이익을 공유하는데 뜻을 같이 함\n유통 질서 및 가격...,A,"시장흐름을 파악하여 변화에 대응할 수 있는 경영능력\n전문성을 갖춘 개술개발팀, 생...",C,"O.E.M , ODM 사 및 해외 수입파트너\n당사의 금형개발, 주조, 연마, 도금...",D,"인건비, 원부자재재료비, 물류비, 광고비\n

In [186]:
df2 = pd.read_csv("data/고객세그먼트.csv")
df2




,고객세그먼트,label
0,"현대건설, 지에스건설 등 건설사\nLH공사, SH공사, 지방 도시공사와 같은 시행사...",A
1,전자부품MLCC(Multi Layer Ceramic Condenser)제조사\n반도...,A
2,주거용 건물 건축주(입주자아파트 등)\n건설부문 외주 및 자재 생산업체(단종업체) ...,B
3,"관공서(행정기관)석면사업부(조사, 감리, 측정)\n제조업(대기업)작업환경측정사업부",B
4,"건설사, 시행사, 설계사무소\n도소매 유통업체\n건축주\nLH공사, SH공사, 지방...",C
...,...,...
499,"욕실 용품 유통업체\n시행사, 시공사, 중소 건설사",C
500,"노즐 및 밸브를 사용하는 기업\n분수, 아파트 정원 공사업체\n환경설비업을 하는 회...",B
501,당사 1단계 목표 : 초중고등학교 \n2단계 목표 : B to C 이며 \n3단계...,C
502,"건설사(민영, 공영), 설계사무소, 시행사\n건축자재 도소매 업체, 인테리어 업체\...",C


In [174]:
total_df = pd.DataFrame()

In [175]:
total_df['기업명'] = df['기업명']

In [176]:
# text컬럼에 모든 컬럼 텍스트 합치기
total_df['text'] = "고객 세그먼트는 "+ df['고객 세그먼트'] + "." +\
            " 가치제안은 " + df['가치제안'] + "." +\
            " 마케팅채널은 " + df['마케팅 채널'] + "." +\
            " 고객관계는 " + df['고객관계'] + "." +\
            " 수익원은 " + df['수익원'] + "." +\
            " 핵심활동은 " + df['핵심활동'] + "." +\
            " 핵심자원은 " + df['핵심자원'] + "." +\
            " 핵심파트너는 " + df['핵심파트너'] + "."\
            " 비용구조는 " + df['비용구조'] + "."

In [14]:
# a = "고객 세그먼트는 "+ df['고객 세그먼트'] + " 가치제안은 " + df['가치제안'] + " 마케팅채널은 " + df['마케팅 채널'] + " 고객관계는 " +\
#     df['고객관계'] + " 수익원은 " + df['수익원'] + " 핵심활동은 " + df['핵심활동'] + " 핵심자원은 " + df['핵심자원'] + " 핵심파트너는 " +\
#     df['핵심파트너'] + " 비용구조는 " + df['비용구조']

In [177]:
total_df

,기업명,text
0,에이앤유디자인그룹건축사사무소,"고객 세그먼트는 현대건설, 지에스건설 등 건설사\nLH공사, SH공사, 지방 도시공..."
1,멜텍스코리아 주식회사,고객 세그먼트는 전자부품MLCC(Multi Layer Ceramic Condense...
2,에스티엑스건설,고객 세그먼트는 주거용 건물 건축주(입주자아파트 등)\n건설부문 외주 및 자재 생산...
3,충청산업보건연구원(주),"고객 세그먼트는 관공서(행정기관)석면사업부(조사, 감리, 측정)\n제조업(대기업)작..."
4,대동산업(주),"고객 세그먼트는 건설사, 시행사, 설계사무소\n도소매 유통업체\n건축주\nLH공사,..."
...,...,...
499,㈜케이바스,"고객 세그먼트는 욕실 용품 유통업체\n시행사, 시공사, 중소 건설사. 가치제안은 소..."
500,진우엔티,"고객 세그먼트는 노즐 및 밸브를 사용하는 기업\n분수, 아파트 정원 공사업체\n환경..."
501,㈜에어붐,고객 세그먼트는 당사 1단계 목표 : 초중고등학교 \n2단계 목표 : B to C...
502,㈜킴스코파쎄,"고객 세그먼트는 건설사(민영, 공영), 설계사무소, 시행사\n건축자재 도소매 업체,..."


# Functions

In [165]:
# 데이터셋 컬럼의 중복을 제외한 2음절 이상의 vocab에 등록된 단어를 list로 만드는 함수 
# 인자로 컬럼 하나의 데이터프레임이 들어가야함. e.g.) df['고객세그먼트'] 

def store_tokenized_words(one_dataset):
    # 총 tokenizing 단어 리스트에 저장
    token_list = []
    for i in range(len(df)):
#         print(tok(one_dataset[i]))
        token_list.append(tok(one_dataset[i]))

    # set에 tokenizing 단어 저장(중복제거)
    token_set = []
    for i in range(len(token_list)):
        for j in range(len(token_list[i])):
            token_set.append(token_list[i][j])
    # set(token_list)

    unique_set = set(token_set)
    unique_list = list(unique_set)

    # 한글자 tokenizing 단어 제외한 후 unique_company_list에 저장
    unique_company_list = []
    for i in unique_list:
        if (i[0] == '▁') & (len(i) >=3):
            unique_company_list.append(i)
    #         print(i)  
        if (i[0] != '▁') & (len(i) >=2):
            unique_company_list.append(i)
    #         print(i)

    # 리스트의 단어에서 '_' 제거
    for i in range(len(unique_company_list)):
        unique_company_list[i] = unique_company_list[i].replace('▁',"")

#     print(unique_company_list)
    return unique_company_list

In [167]:
# 컬럼 내 모든 어절을 word_set에 저장(중복제외)하는 함수
# 인자로 컬럼 하나의 데이터프레임이 들어가야함. e.g.) df['고객세그먼트'] 

def store_segmented_words(one_dataset):
    word_set = set()
    rm_list = ["(", ")", "\n", '"', ","]

    for sentence in one_dataset:
        for rm in rm_list:
            sentence = sentence.replace(rm, " ")

        sentence = sentence.split(" ")
        for word in sentence:
#             print(word)
            word_set.add(word)

    return word_set

In [206]:
# vocab 단어와 컬럼의 어절을 비교하여 vocab에 저장되지 않은 단어를 리스트로 추출하는 함수

def store_untokenized_words(segmented_words, tokenized_words):
    modifying_list = []

    for word in segmented_words:
    #     print(word)
        if word not in tokenized_words:
            modifying_list.append(word)

    return modifying_list

In [207]:
tokenized_words = store_tokenized_word(total_df['text'])
len(tokenized_words)

3222

In [208]:
segmented_words = store_segmented_words(total_df['text'])
len(segmented_words)

22793

In [209]:
untokenized_words = store_untokenized_words(segmented_words, tokenized_words)
len(untokenized_words)

20853

In [210]:
untokenized_words

['',
 '명절날',
 '최적화를',
 '입소문',
 '설계사무소’',
 '비용절감을',
 '매출의',
 '중도매인',
 '포함됩니다.',
 '홍보와',
 '부동산금융인',
 '얻어내기',
 '디지털사이니지',
 '롯데건설外',
 '끝났다.',
 '외벽마감업체',
 '코닉글로리',
 '도정업자',
 '네트워킹',
 '중요한바',
 '향상된',
 '회사임직원',
 '감각으로',
 '건축주[실제',
 '시공시',
 '창의적이고',
 '10%',
 '바로는',
 '친화경',
 '한국실내건축가협회',
 'SERCH하여',
 '미만은',
 '우선시',
 '회사나',
 '하자처리까지',
 '신규업체',
 'internet',
 '소지하고',
 '선사하는',
 '수의계약에',
 '가격대를',
 '차량운영비.',
 '개인:',
 '마트에',
 '할인점을',
 '원우.',
 '전문가들과',
 '시대를',
 '영업관리비',
 '발굴.',
 '서비스상품',
 '신사업',
 '스팩하여',
 '라인이',
 'CE0',
 '공인중개사와의',
 '초청.',
 'R&D',
 '판넬',
 '소개하면',
 '디지털플랫폼',
 '단품',
 '원자재비',
 '기준지표로',
 '우리회사를',
 '표준과',
 '신뢰이며',
 '임대자산',
 '분양시',
 '수요정책에',
 '사립',
 '정보지식',
 '처리비와',
 '수입:',
 '니즈파악',
 '석면조사업',
 '은행자산의',
 '배송시',
 '동등한',
 '가락시장내',
 '은행이라는',
 'SNS체험',
 '준공영체제로서',
 'MDM',
 '납기일정',
 '가공품',
 '호재를',
 '마케팅능력을',
 '핵심활동',
 '한국판',
 'World',
 '건강증진을',
 '상가시설',
 '비회전제품의',
 '지자체와',
 'SNS광고',
 'APP내에서',
 '발휘하여',
 '이용과정에서',
 '고정자산의',
 '자문',
 'ETRI',
 '정부기관',
 '애사심..',
 '기업으로',
 '개선한다',
 '설계사를통한',
 '공급계획',
 '

In [221]:
# untokenized_words_df 데이터프레임 변환
untokenized_words_df = pd.DataFrame(untokenized_words, columns=["untokenized_word"])

In [222]:
# csv로 저장
untokenized_words_df.to_csv("data/untokenized_words_df.csv", encoding='utf-8-sig')

In [143]:
tok('존근당')

['▁존', '근', '당']

In [140]:
modifying_list = []
for token in unique_list:
    if token not in unique_company_list:
        modifying_list.append(token)
        

In [141]:
len(modifying_list)

1753

In [78]:
modifying_list

['겐',
 '태',
 '쎄',
 '▁어',
 'B',
 '잔',
 '집',
 '기',
 '료',
 '머',
 '축',
 '▁콘',
 '▁즉',
 '▁대',
 '업',
 '▁석',
 '롬',
 '딩',
 '와',
 '션',
 'T',
 '멘',
 'R',
 '칼',
 '피',
 '▁음',
 '▁레',
 '▁조',
 '닉',
 '능',
 '거',
 '▁구',
 '소',
 '싱',
 '순',
 '타',
 '▁테',
 '▁요',
 '심',
 '▁본',
 '휴',
 '”',
 '▁/',
 '믹',
 '확',
 '류',
 '실',
 '샵',
 '웅',
 'g',
 '밀',
 '▁독',
 '즐',
 '식',
 'I',
 '알',
 '▁서',
 '켈',
 '킹',
 '▁최',
 '▁부',
 '▁초',
 'f',
 '두',
 '증',
 '▁면',
 '▁돌',
 '블',
 '▁생',
 '▁병',
 '8',
 '해',
 '여',
 '▁슈',
 '할',
 '형',
 '쉐',
 '끼',
 '▁임',
 '잉',
 '▁두',
 '군',
 '전',
 '▁팀',
 '롭',
 '영',
 '우',
 '희',
 '퓨',
 '서',
 '▁뉴',
 '딜',
 '육',
 '▁귀',
 '멜',
 '뚜',
 '롱',
 '▁폐',
 '▁무',
 'y',
 '텐',
 '컨',
 '▁회',
 '샴',
 'k',
 '▁풍',
 '▁맞',
 '▁착',
 '외',
 '쓰',
 '척',
 '▁청',
 '▁건',
 '납',
 '▁변',
 '▁접',
 'G',
 'C',
 '▁주',
 '모',
 '루',
 '▁벌',
 '▁순',
 '책',
 '▁된',
 '반',
 '젤',
 '▁모',
 '▁위',
 '탁',
 '▁수',
 '북',
 '▁질',
 'x',
 '법',
 '파',
 '▁운',
 '▁새',
 '누',
 '득',
 '케',
 '▁크',
 '▁종',
 '량',
 '선',
 '입',
 '인',
 '틀',
 '▁곡',
 '▁암',
 '울',
 '다',
 '퀘',
 '욕',
 '▁꽃',
 '▁T',
 '완',
 '생'